<a href="https://colab.research.google.com/github/SaidaKarimova9/DSD-Final-Project/blob/main/Final_version_of_the_first_model_DANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision
!pip install numpy pandas matplotlib

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
import os

# Paths to the zip files
zip_paths = [
    '/content/drive/MyDrive/ML-2024/AFE.zip',
    '/content/drive/MyDrive/ML-2024/ExpW.zip'
]

# Extract each zip file
for zip_path in zip_paths:
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall('/content/dataset')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Function
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from PIL import Image

class ResizeAndPad:
    def __init__(self, size, interpolation=Image.BILINEAR):
        self.size = size
        self.interpolation = interpolation

    def __call__(self, img):
        img.thumbnail((self.size, self.size), self.interpolation)
        delta_w = self.size - img.size[0]
        delta_h = self.size - img.size[1]
        padding = (delta_w // 2, delta_h // 2, delta_w - (delta_w // 2), delta_h - (delta_h // 2))
        return transforms.functional.pad(img, padding, fill=0)

class FilteredImageFolder(datasets.ImageFolder):
    def __init__(self, root, transform=None, min_size=32):
        super().__init__(root, transform=transform)
        self.min_size = min_size

    def __getitem__(self, index):
        path, target = self.samples[index]
        sample = self.loader(path)
        if min(sample.size) < self.min_size:
            return None
        if self.transform is not None:
            sample = self.transform(sample)
        return sample, target

# Define transformations
transform = transforms.Compose([
    ResizeAndPad(128),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

train_transform = transforms.Compose([
    ResizeAndPad(128),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

# Paths to the datasets
source_train_path = '/content/dataset/AFE/train'
source_test_path = '/content/dataset/AFE/test'
target_train_path = '/content/dataset/ExpW/train'
target_test_path = '/content/dataset/ExpW/test'

# Load datasets
source_train_dataset = FilteredImageFolder(root=source_train_path, transform=train_transform)
source_test_dataset = FilteredImageFolder(root=source_test_path, transform=transform)
target_train_dataset = FilteredImageFolder(root=target_train_path, transform=train_transform)
target_test_dataset = FilteredImageFolder(root=target_test_path, transform=transform)

# Create data loaders
source_train_loader = DataLoader(source_train_dataset, batch_size=32, shuffle=True)
source_test_loader = DataLoader(source_test_dataset, batch_size=32, shuffle=False)
target_train_loader = DataLoader(target_train_dataset, batch_size=32, shuffle=True)
target_test_loader = DataLoader(target_test_dataset, batch_size=32, shuffle=False)

class ReverseLayerF(Function):
    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha
        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        output = grad_output.neg() * ctx.alpha
        return output, None

class DANN(nn.Module):
    def __init__(self, num_classes):
        super(DANN, self).__init__()
        self.feature = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.25),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.25),

            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.25),
        )

        self.class_classifier = nn.Sequential(
            nn.Linear(512 * 16 * 16, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(True),
            nn.Dropout(0.5),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(True),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes),
        )

        self.domain_classifier = nn.Sequential(
            nn.Linear(512 * 16 * 16, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(True),
            nn.Dropout(0.5),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(True),
            nn.Dropout(0.5),
            nn.Linear(512, 2),
        )

    def forward(self, input_data, alpha):
        input_data = input_data.expand(input_data.data.shape[0], 3, 128, 128)
        feature = self.feature(input_data)
        feature = feature.view(-1, 512 * 16 * 16)
        reverse_feature = ReverseLayerF.apply(feature, alpha)
        class_output = self.class_classifier(feature)
        domain_output = self.domain_classifier(reverse_feature)
        return class_output, domain_output

# Training configuration
num_classes = 7
num_epochs = 20
batch_size = 32
learning_rate = 0.001
alpha = 0.3

# Define the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Create the model
model = DANN(num_classes=num_classes).to(device)

# Define the loss functions and optimizer
criterion_class = nn.CrossEntropyLoss()
criterion_domain = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)


In [ ]:
import torch

def load_checkpoint(checkpoint_path):
    checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    return checkpoint['epoch']

checkpoint_path = '/content/drive/MyDrive/ML-2024/checkpoint.pth'
start_epoch = load_checkpoint(checkpoint_path)
print(f"Resumed from epoch {start_epoch}")


Resumed from epoch 20


In [ ]:
# evaluate the model
def evaluate(model, data_loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs, _ = model(images, alpha=0)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    return accuracy

# Evaluate on source and target test datasets
source_accuracy = evaluate(model, source_test_loader, device)
target_accuracy = evaluate(model, target_test_loader, device)

print(f'Resumed Source Test Accuracy: {source_accuracy * 100:.2f}%')
print(f'Resumed Target Test Accuracy: {target_accuracy * 100:.2f}%')


def save_results(phase, source_accuracy, target_accuracy):
    results = {
        'phase': phase,
        'source_accuracy': source_accuracy,
        'target_accuracy': target_accuracy
    }
    torch.save(results, f'/content/drive/MyDrive/ML-2024/{phase}_results.pth')

# Save the resumed evaluation results
save_results('resumed_final', source_accuracy, target_accuracy)



Resumed Source Test Accuracy: 76.94%
Resumed Target Test Accuracy: 26.13%


In [ ]:
import os

save_dir = '/content/drive/MyDrive/ML-2024/saved_models'

os.makedirs(save_dir, exist_ok=True)


In [ ]:
def save_checkpoint(model, optimizer, epoch, filepath):
    state = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }
    torch.save(state, filepath)


In [ ]:

os.makedirs(save_dir, exist_ok=True)

# Training
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for target_batch, target_labels in target_train_loader:
        target_batch, target_labels = target_batch.to(device), target_labels.to(device)

        optimizer.zero_grad()


        outputs, _ = model(target_batch, alpha=0)
        loss = criterion_class(outputs, target_labels)


        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(target_train_loader)}")

    # Save the oracle model checkpoint
    save_checkpoint(model, optimizer, epoch, f'{save_dir}/oracle_epoch_{epoch+1}.pth')

print("Target Only Training finished.")

# Evaluate on target test set
target_only_accuracy = evaluate(model, target_test_loader, device)
print(f'Oracle Target Test Accuracy: {target_only_accuracy * 100:.2f}%')

# Save final oracle results
save_results('oracle_final', target_only_accuracy)


Epoch [1/20], Loss: 2.456855099588152
Epoch [2/20], Loss: 1.708629955026892
Epoch [3/20], Loss: 1.5219264399380452
Epoch [4/20], Loss: 1.4333691099478352
Epoch [5/20], Loss: 1.3872605704311882
Epoch [6/20], Loss: 1.3509853784475596
Epoch [7/20], Loss: 1.3290659583984845
Epoch [8/20], Loss: 1.3075199122991292
Epoch [9/20], Loss: 1.2905153452508873
Epoch [10/20], Loss: 1.2763189351366413
Epoch [11/20], Loss: 1.260695050019891
Epoch [12/20], Loss: 1.2495153453501482
Epoch [13/20], Loss: 1.2374753599625923
Epoch [14/20], Loss: 1.234949983956833
Epoch [15/20], Loss: 1.2235182180940625
Epoch [16/20], Loss: 1.2145985764481368
Epoch [17/20], Loss: 1.2079134402315943
Epoch [18/20], Loss: 1.1988428696208904
Epoch [19/20], Loss: 1.1940884230237814
Epoch [20/20], Loss: 1.1856541344595533
Target Only Training finished.
Oracle Target Test Accuracy: 65.07%


TypeError: save_results() missing 1 required positional argument: 'target_accuracy'